##  ewf-ext-03-03-03 - Flood hazard

### <a name="service">Service definition

In [1]:
service = dict([('title', 'ewf-ext-03-03-03 - Flood exposure'),
                ('abstract', 'ewf-ext-03-03-03 - Flood exposure'),
                ('id', 'ewf-ext-03-03-03')])

In [2]:
start_year = dict([('id', 'start_year'),
            ('value', '2015'),
            ('title', 'start year'),
            ('abstract', 'start year')])

In [3]:
end_year = dict([('id', 'end_year'),
            ('value', '2019'),
            ('title', 'end_year'),
            ('abstract', 'end_year')])

In [4]:
area_of_interest = dict([('id', 'areaOfInterest'),
                         ('value', 'IberianPeninsula'),
                         ('title', 'Area of the region'),
                         ('abstract', 'Area of the region of interest')])

In [5]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                         ('value', '-1'),
                         ('title', 'WKT Polygon for the Region of Interest (-1 if no crop)'),
                         ('abstract', 'Set the value of WKT Polygon')])

### Parameter Definition

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the Sentinel-1 stack of products' identifiers

In [6]:
input_identifiers = ('GHS_POP_E2015_GLOBE_R2019A_54009_250_V1_0_17_4.tif', 'U2018_CLC2018_V2020_20u1.tif')

**Input references**

This is the Sentinel-1 stack catalogue references

In [7]:
input_references = ('https://catalog.terradue.com/chirps/search?format=atom&uid=chirps-v2.0.2017.01.01','https://catalog.terradue.com/chirps/search?format=atom&uid=chirps-v2.0.2017.01.02') 

**Data path**

This path defines where the data is staged-in. 

In [8]:
data_path = "/workspace/data/Copernicus"

In [48]:
#etc_path = "/application/notebook/etc"
etc_path = "/workspace/Better_3rd_phase/Applications/EXT-03-03-02/ewf-ext-03-03-02/src/main/app-resources/notebook/etc"

In [10]:
#output_folder = ""
output_folder = "/workspace/Better_3rd_phase/Applications/EXT-03-03-03/ewf-ext-03-03-03/src/main/app-resources/notebook/libexec"

In [11]:
temp_folder = 'Temp'

In [12]:
cropped_output_folder = 'Output/Crop'

#### Import Modules

In [13]:
import os
import shutil

import sys
import string
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.wkt import loads
import datetime
import gdal

import pdb
from calendar import monthrange


#### Auxiliary methods

In [43]:
# remove contents of a given folder
# used to clean a temporary folder
def rm_cfolder(folder):
    #folder = '/path/to/folder'
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)
            
def crop_image(input_image, polygon_wkt, output_path, product_type=None):
    
    dataset = None
        
    if input_image.startswith('ftp://') or input_image.startswith('http'):
        try:
            dataset = gdal.Open('/vsigzip//vsicurl/%s' % input_image)
        except Exception as e:
            print(e)
    elif '.nc' in input_image:
        dataset = gdal.Open('NETCDF:' + input_image + ':' + product_type)

    polygon_ogr = ogr.CreateGeometryFromWkt(polygon_wkt)
    envelope = polygon_ogr.GetEnvelope()
    bounds = [envelope[0], envelope[3], envelope[1], envelope[2]]         

    gdal.Translate(output_path, dataset, outputType=gdal.GDT_Int16, projWin=bounds, projWinSRS='EPSG:4326')

    dataset = None

    
def write_output_image(filepath, output_matrix, image_format, data_format, mask=None, output_projection=None, output_geotransform=None, no_data_value=None):
    driver = gdal.GetDriverByName(image_format)
    out_rows = np.size(output_matrix, 0)
    out_columns = np.size(output_matrix, 1)
    
    
    if mask is not None and mask is not 0:
        # TODO: check if output folder exists
        output = driver.Create(filepath, out_columns, out_rows, 2, data_format)
        mask_band = output.GetRasterBand(2)
        mask_band.WriteArray(mask)
        if no_data_value is not None:
            output_matrix[mask > 0] = no_data_value
    else:
        output = driver.Create(filepath, out_columns, out_rows, 1, data_format)
    
    if output_projection is not None:
        output.SetProjection(output_projection)
    if output_geotransform is not None:
        output.SetGeoTransform(output_geotransform)
    
    raster_band = output.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
    raster_band.WriteArray(output_matrix)
    
    if filepath is None:
        print  "filepath"
    if output is None:
        print  "output"
    gdal.Warp(filepath, output, format="GTiff", outputBoundsSRS='EPSG:4326', xRes=output_geotransform[1], yRes=-output_geotransform[5], targetAlignedPixels=True)

    return filepath


def matrix_multiply(mat1, mat2, no_data_value=None):
    #if no_data_value is not None:
        #if not isinstance(mat1, int):
            #mat1[(mat1 == no_data_value)] = 0
        #if not isinstance(mat2, int):
            #mat2[(mat2 == no_data_value)] = 0
    mats_nodata = np.logical_or(mat1 == no_data_value, mat2 == no_data_value)
    mat1 = mat1.astype('float32')
    mat2 = mat2.astype('float32')
    multiply = mat1 * mat2
    multiply = np.where(mats_nodata, no_data_value, multiply)
    return multiply
            

'''def get_matrix_list(image_list, product_type):
    projection = None
    geo_transform = None
    no_data = None
    mat_list = []
    for img in image_list:
        dataset = gdal.Open('NETCDF:' + img + ':' + product_type)
        projection = dataset.GetProjection()
        geo_transform = dataset.GetGeoTransform()
        no_data = dataset.GetRasterBand(1).GetNoDataValue()
        product_array = dataset.GetRasterBand(1).ReadAsArray()
        mat_list.append(product_array)
        dataset = None
    return mat_list, projection, geo_transform, no_data'''

def get_matrix_list(image_list):
    projection = None
    geo_transform = None
    no_data = None
    mat_list = []
    for img in image_list:
        dataset = gdal.Open(img)
        print dataset
        projection = dataset.GetProjection()
        print projection
        geo_transform = dataset.GetGeoTransform()
        no_data = dataset.GetRasterBand(1).GetNoDataValue()
        product_array = dataset.GetRasterBand(1).ReadAsArray()
        mat_list.append(product_array)
        dataset = None
    return mat_list, projection, geo_transform, no_data
    
def write_outputs(product_name, first_date, last_date, averages, standard_deviation, image_format, projection, geo_transform, no_data_value):
    filenames = []
    areaofinterest = area_of_interest['value']
    filenames.append(product_name + '_averages_' + areaofinterest + '_' + first_date + '_' + last_date + '.tif')
    filenames.append(product_name + '_standarddeviation_' + areaofinterest + '_'+ first_date + '_' + last_date + '.tif')

    write_output_image(filenames[0], averages, image_format, gdal.GDT_Int16, None, projection, geo_transform, no_data_value)
    write_output_image(filenames[1], standard_deviation, image_format, gdal.GDT_Int16, None, projection, geo_transform, no_data_value)
    
    return filenames

def write_properties_file(output_name, first_date, last_date):
    
    title = 'Output %s' % output_name
    
    first_date = get_formatted_date(first_date)
    last_date = get_formatted_date(last_date)
    
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (regionOfInterest['value']))
        
def get_formatted_date(date_obj):
    date = datetime.datetime.strftime(date_obj, '%Y-%m-%dT00:00:00Z')
    return date

def reproject_image_to_master ( master, slave, dst_filename, res=None ):

    slave_ds = gdal.Open( slave )
    if slave_ds is None:
        raise IOError, "GDAL could not open slave file %s " \
            % slave
    slave_proj = slave_ds.GetProjection()
    slave_geotrans = slave_ds.GetGeoTransform()
    data_type = slave_ds.GetRasterBand(1).DataType
    n_bands = slave_ds.RasterCount

    master_ds = gdal.Open( master )
    if master_ds is None:
        raise IOError, "GDAL could not open master file %s " \
            % master
    master_proj = master_ds.GetProjection()
    master_geotrans = master_ds.GetGeoTransform()
    w = master_ds.RasterXSize
    h = master_ds.RasterYSize
    
    if res is not None:
        master_geotrans[1] = float( res )
        master_geotrans[-1] = - float ( res )
    
    dst_ds = gdal.GetDriverByName('GTiff').Create(dst_filename, w, h, n_bands, data_type)
    
    dst_ds.SetGeoTransform( master_geotrans )
    dst_ds.SetProjection( master_proj)
    
    gdal.ReprojectImage( slave_ds, dst_ds, slave_proj,
                         master_proj, gdal.GRA_NearestNeighbour)
    
    dst_ds = None  # Flush to disk
    
    return dst_filename

def project_coordinates(file, dst_filename):
    input_raster = gdal.Open(file)
    output_raster = dst_filename 
    gdal.Warp(output_raster,input_raster,dstSRS='EPSG:4326')
    
def get_pixel_weights(mat):
    urban_fabric=[111.,112.]
    industrial_commercial_transport_units=[121.,122.,123.,124.]
    mine_dump_construction_sites=[131.,132.,133.]
    artificial_areas=[141.,142.]
    arable_land=[211.,212.,213.]
    permanent_crops=[221.,222.,223.]
    pastures=[231.]
    agricultural_areas=[241.,242.,243.,244.]
    forest=[311.,312.,313.]
    vegetation_associations=[321.,322.,323.,324.]
    little_no_vegetation=[331.,332.,333.,334.,335.]
    inland_wetlands=[411.,412.]
    coastal_wetlands=[421.,422.,423.]
    inland_waters=[511.,512.]
    marine_waters=[521.,522.,523.]

    exposure_dictionary = dict()
    exposure_dictionary[1.0] = urban_fabric
    exposure_dictionary[0.5] = industrial_commercial_transport_units + arable_land + permanent_crops
    exposure_dictionary[0.3] = mine_dump_construction_sites + agricultural_areas
    exposure_dictionary[0.0] = artificial_areas + marine_waters
    exposure_dictionary[0.4] = pastures
    exposure_dictionary[0.1] = forest + vegetation_associations + little_no_vegetation + inland_wetlands + coastal_wetlands + inland_waters

    rows = mat.shape[0]
    cols = mat.shape[1]

    for i in range(0, rows):
        for j in range(0, cols):
            for exposure, value_list in exposure_dictionary.iteritems():
                for value in value_list:
                    if mat[i,j] == value:
                        mat[i,j] = exposure
    return mat



In [44]:
if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

In [45]:
area_of_interest['value'], start_year['value'], end_year['value']

('IberianPeninsula', '2015', '2019')

#### Workflow

#### Update AOI if crop not needed

In [46]:
input_products = ('GHS_POP_E2015_GLOBE_R2019A_54009_250_V1_0_17_4.tif', 'U2018_CLC2018_V2020_20u1.tif')

In [50]:
first_year = start_year['value']
last_year = end_year['value']
product_path_name = output_folder
projection = None
geo_transform = None
no_data = None
areaofinterest = area_of_interest['value']

if input_products[0] >=0:    
        file_list = [os.path.join(etc_path, filename) for filename in input_products]
        print file_list[0]
        mat_list, projection, geo_transform, no_data = get_matrix_list(file_list)
        print mat_list

        flood_frequency = mat_list[0]
        flood_exposure = mat_list[1]
        flood_hazard = matrix_multiply(flood_frequency,flood_exposure, no_data)
        
        file = write_output_image(os.path.join(product_path_name , 'FEI_' + areaofinterest + '_GHS_' + first_year + '_CLC_'+ last_year + '.tif'), flood_hazard, 'GTiff', gdal.GDT_Float32, None, projection, geo_transform, no_data)
        firstdate_obj = datetime.datetime.strptime(first_year, "%Y").date()
        lastdate_obj = datetime.datetime.strptime(last_year, "%Y").date()
        
else:
        print "error" + file_list


/workspace/Better_3rd_phase/Applications/EXT-03-03-02/ewf-ext-03-03-02/src/main/app-resources/notebook/etc/GHS_POP_E2015_GLOBE_R2019A_54009_250_V1_0_17_4.tif
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f23fab829c0> >
PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.017453292519943295]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f23fab829c0> >
PROJCS["ETRS_1989_LAEA",GEOGCS["ETRS89",DATUM["European_Terrestrial_Reference_System_1989",SPHEROID["GRS 1980",6378137,298.2572221010042,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6258"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4258"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_cent

ValueError: operands could not be broadcast together with shapes (4000,4000) (46000,65000) 

In [39]:
if input_products[0] >=0:    
    if regionOfInterest['value'] == '-1':

        #dataset = gdal.Open('/vsigzip//vsicurl/%s' % gpd_final.iloc[0]['enclosure'])
        dataset = gdal.Open(file)

        geoTransform = dataset.GetGeoTransform()

        minx = geoTransform[0]
        maxy = geoTransform[3]
        maxx = minx + geoTransform[1] * dataset.RasterXSize
        miny = maxy + geoTransform[5] * dataset.RasterYSize

        regionOfInterest['value'] = 'POLYGON(({0} {1}, {2} {1}, {2} {3}, {0} {3}, {0} {1}))'.format(minx, maxy, maxx, miny)

        dataset = None

    regionofinterest = regionOfInterest['value']
    write_properties_file(file, firstdate_obj, lastdate_obj)


RuntimeError: not a string

#### Remove temporay files and folders

In [ ]:
try:
    shutil.rmtree(temp_folder)
    shutil.rmtree(cropped_output_folder)
except OSError as e:
    print("Error: %s : %s" % (temp_folder, e.strerror))
    print("Error: %s : %s" % (cropped_output_folder, e.strerror))